In [1]:
import pandas as pd
import pyomo.environ as pyo
import networkx as nx
import matplotlib.pyplot as plt
import random
from random import sample

In [2]:
nodes = pd.read_pickle("../../data/mock/mock_nodes.pkl")
channels = pd.read_pickle("../../data/mock/mock_channels.pkl")

In [3]:
nodes.to_csv("../../data/mock/mock_nodes.csv")
channels.to_csv("../../data/mock/mock_channels.csv")

In [4]:
## Note that the following are arbitrary policies, that can be changed as needed.
## Deal with multi-edges (aka multiple channels between two peers):
## - average rate fee
## - average base fee,
## - average capacity
## - keep one of the two channel ids
aggregation_dict = {
    "channel_id": "first",
    "rate_fee": "mean",
    "base_fee": "mean",
    "capacity": "sum"
}
channels.reset_index(inplace=True)
channels = channels.groupby(["node1_pub", "node2_pub"]).agg(aggregation_dict)
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

In [5]:
## To delete from [outgoing] and [incoming] lists the multi-edge
for i in nodes.index:
    chan = []
    for l in nodes.loc[i, "outgoing_channels"]:
        if l in channels.index:
            chan.append(l)
    nodes.at[i, "outgoing_channels"] = chan

for i in nodes.index:
    chan = []
    for l in nodes.loc[i, "incoming_channels"]:
        if l in channels.index:
            chan.append(l)
    nodes.at[i, "incoming_channels"] = chan


**The execution of this chunck highlights how the channel directions is often wrong, even though the peers seem to be correct. There's probably an issue in the detection of the directed relationship.

- [ ] solve this issue

In [27]:
# Add to the nodes dataset the list of peers (nodes who have a channel with the N node)
for n in nodes.index:
    peers = []
    for c in nodes.loc[n, "outgoing_channels"]:
        source, destination = channels[channels["channel_id"]==c].index[0]
        print(f"For node {n}, the channel {c} is {source} --> {destination}")
        #try:
        #    assert source == n
        #    peers.append(destination)
        #except AssertionError as e:
        #    print(f"{source} is NOT equal to {n} for channel {c}")
    print(peers)
 

For node FNRZTCUAZH, the channel 009566356891778588 is FNRZTCUAZH --> EZOBHZBFPU
For node FNRZTCUAZH, the channel 002022819420050336 is FNRZTCUAZH --> IUTMEIARAL
For node FNRZTCUAZH, the channel 111742423300785317 is FNRZTCUAZH --> RMCLDHZQZI
For node FNRZTCUAZH, the channel 308829925196470866 is FNRZTCUAZH --> HAVOHDULRN
For node FNRZTCUAZH, the channel 557332762177297363 is FNRZTCUAZH --> LHNDZFRFEI
[]
For node FQOCLOMAZN, the channel 075763090018637224 is VQPGEVSSQH --> FQOCLOMAZN
For node FQOCLOMAZN, the channel 823213265093346966 is IDZCVBUINF --> FQOCLOMAZN
For node FQOCLOMAZN, the channel 448486103730170488 is FQOCLOMAZN --> VTRQTZMDZB
For node FQOCLOMAZN, the channel 088754794959294646 is UENFNHHIMC --> FQOCLOMAZN
For node FQOCLOMAZN, the channel 796405449656593698 is CPZVPUBAAH --> FQOCLOMAZN
For node FQOCLOMAZN, the channel 910051561595375963 is FQOCLOMAZN --> PGHEECSUVU
For node FQOCLOMAZN, the channel 579406143782824321 is FQOCLOMAZN --> OQAPDLNBGO
For node FQOCLOMAZN, the 

In [17]:
nodes[0:2]

,alias,addresses,outgoing_channels,incoming_channels,demand
pub_key,,,,,
FNRZTCUAZH,FNRZTCUAZH.com,1,"[009566356891778588, 002022819420050336, 11174...","[INV009566356891778588, INV002022819420050336,...",0
FQOCLOMAZN,FQOCLOMAZN.com,1,"[075763090018637224, 823213265093346966, 44848...","[INV075763090018637224, INV823213265093346966,...",0


In [18]:
channels[0:2]

channel_id  rate_fee  base_fee capacity
node1_pub  node2_pub                                                     
APBUDNNRZG IPFVTVBUIQ  INV764532721223089716  0.000004     0.415  3886451
           QBFNTCSMPM  INV181909812366737128  0.000003     0.525  8925058

In [6]:
#nodes.reset_index(inplace=True)
#G = nx.DiGraph()
#
## Add nodes to the graph
#for _, node in nodes.iterrows():
#    G.add_node(node['pub_key'], alias=node['alias'])
#
## Add edges to the graph
#for _, edge in channels.iterrows():
#    G.add_edge(edge['node1_pub'], edge['node2_pub'], capacity=edge['capacity'], base_fee=edge['base_fee'], #perc_fee=edge['rate_fee'])
#
## Draw the graph
#pos = nx.spring_layout(G)  # positions for all nodes
#nx.draw(G, pos, with_labels=True, node_size=2000, node_color='skyblue', font_size=10, font_color='black', #font_weight='bold')
#
## Add edge labels
#edge_labels = {(u, v): f"{d['capacity']}" for u, v, d in G.edges(data=True)}
#nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
#
#plt.title('Network Graph')
#plt.show()
#nodes.set_index("pub_key", inplace=True)

In [7]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=[(channels.loc[i, "node1_pub"], channels.loc[i, "node2_pub"]) for i in channels.index])

In [8]:

def create_demand(pd_object: pd.DataFrame) -> pd.DataFrame:
    """
    This function assigns the role of sender and receiver to
    two random nodes in the network
    :param pd_object: nodes dataframe
    :return: nodes dataset with demand column
    """
    #random.seed(874631)
    counterparties = sample(pd_object.index.to_list(), 2)
    sender = counterparties[0]
    receiver = counterparties[1]
    # Amounts in millisat (aka 10'000'000 is 10'000 sats)
    amount = 10000 #random.randint(a=10000000, b=30000000)

    print(
        f"Transaction of {amount} sats from {pd_object[pd_object.index == sender]['alias'].item()} to {pd_object[pd_object.index == receiver]['alias'].item()}.")

    pd_object["demand"] = 0
    pd_object.loc[pd_object.index == sender, "demand"] = -amount
    pd_object.loc[pd_object.index == receiver, "demand"] = amount

    return pd_object


nodes = create_demand(nodes)



Transaction of 10000 sats from ECSRSZILLM.com to GGSUARQMEQ.com.


In [9]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals, bounds=(0, max(nodes["demand"])))

In [10]:
channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)

In [11]:
def objective_function(model: pyo.ConcreteModel):
    return sum(model.a[i] * channels.loc[i, "rate_fee"] for i in model.CHANNELS) + sum(model.x[i] * channels.loc[i, "base_fee"] for i in model.CHANNELS)

model.totalCost = pyo.Objective(rule=objective_function(model), sense=pyo.minimize)

In [16]:
for i, j in model.x:
    print(model.x[i, j].value)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [12]:
# Single path constrain
def single_path_rule(model: pyo.ConcreteModel, n):
    outgoing_edges = nodes.loc[n, "outgoing_channels"]
    incoming_edges = nodes.loc[n, "incoming_channels"]
    
    return sum(model.x[n, j].value for j in outgoing_edges) == 1 and sum(model.x[i, j].value for i, j in incoming_edges) == 1

#model.SinglePathConstraint= pyo.Constraint(model.NODES, rule=single_path_rule)

ERROR: Rule failed when generating expression for Constraint
SinglePathConstraint with index FNRZTCUAZH: IndexError: only integers, slices
(`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays
are valid indices
ERROR: Constructing component 'SinglePathConstraint' from data=None failed:
        IndexError: only integers, slices (`:`), ellipsis (`...`),
        numpy.newaxis (`None`) and integer or boolean arrays are valid indices


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
def capacity_constraint(model: pyo.ConcreteModel, a, b):
    return model.a[(a, b)] <=  channels.loc[(a, b), "capacity"] * (1 if channels.loc[(a,b), "base_fee"]<1 else model.x[(a, b)])

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

In [ ]:
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    InFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'incoming_channels'])
    OutFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'outgoing_channels'])
    return  InFlow + nodes.loc[n, "demand"] == OutFlow

model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)        

In [ ]:
opt = pyo.SolverFactory('cbc')
results = opt.solve(model, tee=True)

if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('\nOptimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
    print('\nFeasible but not proven optimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    raise Exception("The model is infeasible")
else:
    print('\nSolver Status: ',  results.solver.status)
    raise Exception(results.solver.status)

print('\nObject function value = ', model.Objective())



In [ ]:
ris = []
for i,v in model.a.extract_values().items():
    if v != 0:
        ris.append((i, v))

print(ris)


In [ ]:
from decimal import Decimal
DF = pd.DataFrame()
c = 0
for index, value in model.a.extract_values().items():
    DF.loc[c, "amount"] = Decimal(value)
    DF.loc[c, "source"] = index[0]
    DF.loc[c, "destination"] = index[1]
    c += 1

In [ ]:
DF[DF["amount"]>0]

In [ ]:
nodes.reset_index(inplace=True)
DF_nodes = nodes[nodes["pub_key"].isin(DF.loc[DF["amount"] > 0, "source"].tolist()) | nodes["pub_key"].isin(DF.loc[DF["amount"] > 0, "destination"].tolist())]
nodes.set_index("pub_key", inplace=True)

DF_nodes